In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PacTimeOrig.controllers import simulator as sim
from PacTimeOrig.controllers import JaxMod as jm
from PacTimeOrig.controllers import models as mods

from PacTimeOrig.controllers import utils as ut
from PacTimeOrig.data import scripts
from PacTimeOrig.controllers.Scripts import SimulationScript as simrun


cfgparams={'area':'dACC'}
cfgparams['session']=1
cfgparams['subj']='H'
cfgparams['rbfs']=[15,20,28]
cfgparams['models']=['p','pv','pf','pvi','pif','pvf']
cfgparams['gpscaler']=[1,3,8]
cfgparams['restarts']=3
cfgparams['slack']=False
cfgparams['trials']=10 #TODO use this to select N integer trials
cfgparams['ngains']=[1,2,2,3,3,3] #TODO: Use this to generate gains for simulation
cfgparams['opttype']=['first','second']# Adam and Lbfgs
cfgparams['opttype']=['first']# Adam and Lbfgs

cfgparams['confusiontest']=True# Adam and Lbfgs

simrun.simulate_mp(cfgparams)
simrun.simulate(cfgparams)

Simulate N trials, N-RBFs, N-optimizations, N-runs per, N swtiching types, N random gains, Model type


DiffE+ Adam

In [ ]:
#Set diff e loss
loss_function = mods.create_loss_function_pvi(ut.generate_rbf_basis)

#Set grad  loss
grad_loss = ut.compute_loss_gradient(loss_function)

#Runn diffEV
(L1_opt, L2_opt), best_params, best_loss = mods.outer_optimization_pvi(inputs, mods.inner_optimization_pvi, loss_function, grad_loss,maxiter=1,tolerance=1e-3,opttype='global')

#initialize parameters for ADAM
params=(jnp.array(best_params[0]),jnp.array(best_params[1]),jnp.array(L1_opt),jnp.array(L2_opt))

#setup ADam optimizer
# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-2,slack_model=False,optimizer='amsgrad')

# Number of optimization steps
num_steps = 15000

loss_function = jm.create_loss_function_pvi(ut.generate_rbf_basis,inputs['num_rbfs'],ut.generate_smoothing_penalty,lambda_reg=0.00002)

grad_loss = ut.compute_loss_gradient(loss_function)



# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs,ctrltype='pvi',slack_model=False)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")
        

# Jax - trust region 

In [ ]:
loss_function = jm.create_loss_function_pvi_lbfgs(ut.generate_rbf_basis,num_rbfs)
grad_loss = ut.compute_loss_gradient(loss_function)
hess_loss = ut.compute_hessian(loss_function)


pp,best_params_flat, best_loss = jm.outer_optimization_lbfgs(inputs, loss_function, grad_loss, hess_loss,randomize_weights=False,ctrltype='pvi',maxiter=3000,tolerance=1e-3,optimizer='trust',slack_model=False)

L1_fit=pp[0]
L2_fit=pp[1]

weights = pp[2]
widths = pp[3]
wtout=ut.generate_sim_switch(inputs, widths, weights)

shift=np.vstack((wtout[0],wtout[1]))

pred_out=sim.controller_sim_pvi_post(shift,outputs['x'][:,:2],outputs['x'][:,2:],tdat['pry1_pos'],tdat['pry1_vel'],tdat['pry2_pos'],tdat['pry2_vel'],L1_fit,L2_fit,A=None,B=None)

plt.plot(tmp,outputs['x'][:,:2],linewidth=4)
plt.plot(tmp,pred_out['x'][:,:2],'--',linewidth=4)
plt.show()

plt.plot(tmp,outputs['shift'].transpose(),linewidth=4)
plt.plot(tmp,shift.transpose(),'--',linewidth=4)
plt.show()

Adam Only

In [ ]:
loss_function = jm.create_loss_function_pvi(ut.generate_rbf_basis,inputs['num_rbfs'],ut.generate_smoothing_penalty,lambda_reg=0.00002)

grad_loss = ut.compute_loss_gradient(loss_function)

params=jm.initialize_parameters(inputs, ctrltype='pvi', slack_model=False)


# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-2,slack_model=False,optimizer='amsgrad')

# Number of optimization steps
num_steps = 30000

# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs,ctrltype='pvi',slack_model=False)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")
        
        
        
#transform paramteres to correct domain
L1_fit=jnp.log(1+jnp.exp(params[2]))
L2_fit=jnp.log(1+jnp.exp(params[3]))

weights = params[0]
widths = params[1]
wtout=ut.generate_sim_switch(inputs, widths, weights,slack_model=False)

shift=np.vstack((wtout[0],wtout[1]))

pred_out=sim.controller_sim_pvi_post(shift,outputs['x'][:,:2],outputs['x'][:,2:],tdat['pry1_pos'],tdat['pry1_vel'],tdat['pry2_pos'],tdat['pry2_vel'],L1_fit,L2_fit,A=None,B=None)

plt.plot(tmp,outputs['x'][:,:2],linewidth=4)
plt.plot(tmp,pred_out['x'][:,:2],'--',linewidth=4)
plt.show()

plt.plot(tmp,outputs['shift'].transpose(),linewidth=4)
plt.plot(tmp,shift.transpose(),'--',linewidth=4)
plt.show()